In [1]:
# transaction class
class Transaction:
    def __init__(self, amount, currency, category, date, note=""):
        self.amount = amount # amount of money
        self.category = category # category of the transaction
        self.currency = currency # currency of the transaction
        self.date = date # transaction date
        self.note = note # optional, note for the transaction

    # print the transaction
    def __str__(self):
        return f"Amount: {self.amount}, Currency: {self.currency}, Category: {self.category}, Date: {self.date}, Note: {self.note}"

In [2]:
# Quick sort algorithm
# sort the list of transactions by date
def quicksort(my_list, first_index, last_index):
    if first_index < last_index:
        partition_index = partition(my_list, first_index, last_index)
        quicksort(my_list, first_index, partition_index - 1)
        quicksort(my_list, partition_index + 1, last_index)

def partition(my_list, first_index, last_index):
    pivot = my_list[first_index]
    left_pointer = first_index + 1
    right_pointer = last_index
    while True:
        while my_list[left_pointer].date < pivot.date and left_pointer < last_index:
            left_pointer += 1
        while my_list[right_pointer].date > pivot.date and right_pointer >= first_index:
            right_pointer -= 1
        if left_pointer >= right_pointer:
            break
        my_list[left_pointer], my_list[right_pointer] = my_list[right_pointer], my_list[left_pointer]
    my_list[first_index], my_list[right_pointer] = my_list[right_pointer], my_list[first_index]
    return right_pointer

In [3]:
# AccountBook class
# This class is used to store a list of transactions
# Provide methods to add, remove, and query transactions
class AccountBook:
    def __init__(self):
        self.transactions = [] # list of transactions

    # add a transaction to the account book
    def add_transaction(self, transaction):
        self.transactions.append(transaction)

        # sort the transactions by date
        # use quicksort function to sort the transactions by date
        quicksort(self.transactions, 0, len(self.transactions) - 1)

    # remove a transaction from the account book
    def remove_transaction(self):
        # if there is no transaction, print a message and return
        if len(self.transactions) == 0:
            print("There is no transaction in the account book. Please add a transaction first before removing.")
            return

        # print all transactions with an index number for user to choose later
        print("\nCurrent transactions:")
        for i, transaction in enumerate(self.transactions, 1):
            print(f"{i}. {transaction}")

        while True:
            try: # if the input is not a number, print an error message
                # ask user to input the index number of the transaction to remove
                index = int(input("\nEnter the number of transaction to remove: "))
                
                # check if the input is valid
                if 1 <= index <= len(self.transactions):
                    # remove the transaction the user chose
                    removed_transaction = self.transactions.pop(index - 1)
                    print(f"\nRemoved transaction: {removed_transaction}")
                    return
                else:
                    print("Invalid transaction number. Please try again.") # print error message if the input is invalid
            except ValueError:
                print("Please enter a valid number.")

    # query transactions based on category, start date, and end date
    def query_transactions(self, category=None, start_date=None, end_date=None): # default values are None
        result = self.transactions # initialize the result as all transactions
        # if category is not None, filter the transactions by category
        if category != None:
            result = [r for r in result if r.category == category]

        # if start_date is not None but end_date is None, filter the transactions by start_date
        if start_date != None and end_date == None:
            start_index = self.binary_search_start(start_date, result)
            end_index = None
            result = result[start_index:end_index]

        # if start_date is None but end_date is not None, filter the transactions by end_date
        elif start_date == None and end_date != None:
            start_index = None
            end_index = self.binary_search_end(end_date, result)
            result = result[start_index:end_index]

        # if both start_date and end_date are not None, filter the transactions by both start_date and end_date
        elif start_date != None and end_date != None:
            start_index = self.binary_search_start(start_date, result)
            end_index = self.binary_search_end(end_date, result)
            result = result[start_index:end_index]

        return result # return the filtered transactions

    # binary search to find the first transaction on or after the target date
    def binary_search_start(self, target_date, target_list):
        first = 0
        last = len(target_list) - 1

        while first <= last:
            middle = (first + last) // 2
            if target_list[middle].date >= target_date:
                last = middle - 1
            else:
                first = middle + 1
        return first

    # binary search to find the first transaction after the target date
    def binary_search_end(self, target_date, target_list):
        first = 0
        last = len(target_list) - 1

        while first <= last:
            middle = (first + last) // 2
            if target_list[middle].date > target_date:
                last = middle - 1
            else:
                first = middle + 1
        return first


In [4]:
# install requests modules if not installed
!pip install requests

In [5]:
# exchange rates api call
# only works for euro as base currency
import requests 
# get the exchange rates from the api with access key
response = requests.get('http://api.exchangeratesapi.io/v1/latest?access_key=635c278b3a0984114ce06fa3c4f1ef5e')
exchange_rates = response.json()['rates'] # dictionary of exchange rates of all currencies with respect to euro
currency_codes = list(exchange_rates.keys()) + ['EUR'] # list of currency codes

In [6]:
# currency_converter class
# convert different currency to euro dollar
class EuroConverter:
    def __init__(self, currency, exchange_rates):
        self.rates = exchange_rates[currency]

    def euro_convert(self, amount):
        euro_amount = amount / self.rates
        return euro_amount

In [7]:
# import datetime module to work with date
from datetime import datetime

# create an account book
account_book = AccountBook()


# check if the date is in the correct format
# the date format should be YYYY-MM-DD
# make sure the date input in main functin is in the correct format
def input_date(date):
    while True:
        try:
            date_input = input(date)
            return datetime.strptime(date_input, "%Y-%m-%d")
        except ValueError:
            print("Invalid date format. Please enter the date in YYYY-MM-DD format.")

# main function
def main():
    while True:
        # provide a menu for the user to choose an option
        print("\n1. Add Transaction")
        print("2. Remove Transaction")
        print("3. View Transactions")
        print("4. Convert Currency into Euro")
        print("5. Exit")

        # ask the user to choose an option from the menu
        choice = input("Choose an option from the menu: ")

        # if the user input is 1, add a transaction to the account book
        if choice == "1":
            while True:
                # make sure the amount is a valid number
                try:
                    amount = float(input("Enter amount: ")) # ask the user to input the amount of the transaction
                    break
                # if the amount is not a number, print an error message to ask the user to enter a valid number
                except ValueError:
                    print("Invalid amount. Please enter a valid number.")

            # make sure the currency code is valid
            while True:
                currency = input("Enter currency code (e.g., USD): ") # ask the user to input the currency code of the transaction
                if currency in currency_codes:
                    break
                # if the currency code is not valid, print an error message to ask the user to enter a valid currency code
                else:
                    print("Invalid currency code. Please enter a valid currency code.")

            category = input("Enter category: ") # ask the user to input the category of the transaction
            date = input_date("Enter date (YYYY-MM-DD): ") # ask the user to input the date of the transaction
            note = input("Enter note (optional): ") # ask the user to input the note of the transaction, but the note is optional
            transaction = Transaction(amount, currency, category, date, note) # create a transaction object with the user input
            account_book.add_transaction(transaction) # add the transaction to the account book

            # print a message to show the transaction is added, and show all transactions in the account book to the user 
            print("\nTransaction added.")
            print("All transactions:")
            for t in account_book.transactions:
                print(t)

        # if the user input is 2, remove a transaction from the account book
        elif choice == "2":
            account_book.remove_transaction() # use the remove_transaction method to remove a transaction from the account book
            print("Current transactions after removing are showed below:") # show all transactions after removing to the user
            for t in account_book.transactions:
                print(t)

        # if the user input is 3, view transactions in the account book
        # user can filter the transactions by category, start date, and end date
        elif choice == "3":
            # user can choose to filter the transactions by category or not
            category = input("Enter category to filter (or press enter to skip): ") or None

            while True:
                start_date_filter = input("Do you want to filter by start date? (y/n): ")
                # if the user input is y, ask the user to input the start date
                if start_date_filter.lower() == "y":
                    start_date = input_date("Enter start date (YYYY-MM-DD): ")
                    break
                elif start_date_filter.lower() == "n":
                    start_date = None
                    break
                # if the user input is not y or n, print an error message to ask the user to enter y or n
                else:
                    print("Please answer y or n.")

            while True:
                end_date_filter = input("Do you want to filter by end date? (y/n): ")
                # if the user input is y, ask the user to input the end date
                if end_date_filter.lower() == "y":
                    end_date = input_date("Enter end date (YYYY-MM-DD): ")
                    break
                elif end_date_filter.lower() == "n":
                    end_date = None
                    break
                # if the user input is not y or n, print an error message to ask the user to enter y or n
                else:
                    print("Please answer y or n.")

            # use the query_transactions method to filter the transactions based on the user input
            transactions = account_book.query_transactions(category, start_date, end_date)
            # print the filtered transactions to the user
            print("\nThe transactions are showed below:")
            for t in transactions:
                print(t)

        # if the user input is 4, convert the amount of money from the original currency to euro
        elif choice == "4":
            # make sure the amount is a valid number
            while True:
                try:
                    amount = float(input("Enter amount: ")) # ask the user to input the amount of the transaction
                    break
                # if the amount is not a number, print an error message to ask the user to enter a valid number
                except ValueError:
                    print("Invalid amount. Please enter a valid number.")

            # make sure the currency code is valid
            while True:
                from_currency = input("Enter from currency code (e.g., USD): ")
                # if the currency code is valid, convert the amount to euro
                try:
                    converter = EuroConverter(from_currency, exchange_rates)
                    converted_amount = converter.euro_convert(amount)
                    print(f"\nConverted amount: {converted_amount:.2f} EUR")
                    break
                # if the currency code is invalid, print an error message to ask the user to enter a valid currency code again
                except KeyError:
                    print("Invalid currency code. Please enter a valid currency code.")

        # if the user input is 5, exit the program
        elif choice == "5":
            print("\nExit the expense tracker")
            break

        # if the user input is not between 1 and 5, print an error message to ask the user to enter a valid choice
        else:
            print("Invalid choice. Please enter a choice between 1 to 5.")

In [8]:
# run the main function
if __name__ == "__main__":
    main()


1. Add Transaction
2. Remove Transaction
3. View Transactions
4. Convert Currency into Euro
5. Exit

Transaction added.
All transactions:
Amount: 100.0, Currency: EUR, Category: food, Date: 2024-12-01 00:00:00, Note: 

1. Add Transaction
2. Remove Transaction
3. View Transactions
4. Convert Currency into Euro
5. Exit

Transaction added.
All transactions:
Amount: 100.0, Currency: EUR, Category: food, Date: 2024-12-01 00:00:00, Note: 
Amount: 200.0, Currency: EUR, Category: food, Date: 2024-12-05 00:00:00, Note: lunch

1. Add Transaction
2. Remove Transaction
3. View Transactions
4. Convert Currency into Euro
5. Exit

Transaction added.
All transactions:
Amount: 100.0, Currency: EUR, Category: food, Date: 2024-12-01 00:00:00, Note: 
Amount: 200.0, Currency: EUR, Category: food, Date: 2024-12-05 00:00:00, Note: lunch
Amount: 300.0, Currency: USD, Category: travel, Date: 2024-12-06 00:00:00, Note: 

1. Add Transaction
2. Remove Transaction
3. View Transactions
4. Convert Currency into Eur